In [ ]:
import json
import numpy as np
import sklearn
import sklearn.metrics
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import datetime
# import snap
import random
from numpy.random import choice
import numpy as np
from collections import defaultdict
import os
from tqdm import tqdm_notebook as tqdm
import playlist
reload(playlist)

In [ ]:
playlist_list = playlist.get_playlist_list()

In [ ]:
len(playlist_list)

In [ ]:
import sys
sys.getsizeof(playlist_list)

In [ ]:
track_embedding_df = pd.read_csv("word2vecf-features.txt", skiprows=1, header=None, sep=' ')
track_embedding_df = track_embedding_df.set_index(0)
track_embedding_df = track_embedding_df.drop(columns=[track_embedding_df.shape[1]])
track_embedding_df.columns

In [ ]:
track_embedding_df.shape

In [ ]:
track_embedding_df.head()

In [ ]:
# track_embedding_df.loc['spotify:track:0uqPG793dkDDN7sCUJJIVC']

In [ ]:
# playlist_list[0]['tracks'][0]['track_uri']
# ((playlist_list[0]['num_tracks']))

In [ ]:
cached_track_embedding = {}

def get_track_embedding(track_uri):
    if track_uri in cached_track_embedding:
        return cached_track_embedding[track_uri]
    embedding = track_embedding_df.loc[track_uri].values
    norm_embedding = np.linalg.norm(embedding)
    cached_track_embedding[track_uri] = (embedding, embedding/norm_embedding)
    return cached_track_embedding[track_uri]

In [ ]:
get_track_embedding('spotify:track:0uqPG793dkDDN7sCUJJIVC')[0].shape

In [ ]:
cached_playlist_embedding = {}

def get_playlist_embedding(playlist_id):
    # check if already in cache 
    if playlist_id in cached_playlist_embedding:
        return cached_playlist_embedding[playlist_id]

    embedding_list = []
    for i in range(playlist_list[playlist_id]['num_tracks']):
        track_uri = playlist_list[playlist_id]['tracks'][i]['track_uri']
    #     print(type(track_df.loc[track_uri]))
        embedding_list.append(get_track_embedding(track_uri)[0])
    embedding = np.mean(embedding_list, axis = 0)
    norm_embedding = np.linalg.norm(embedding)
    # put in the cache
    cached_playlist_embedding[playlist_id] = (embedding, embedding/norm_embedding)
    return cached_playlist_embedding[playlist_id]

In [ ]:
for playlist_id in tqdm(range(len(playlist_list))):
    get_playlist_embedding(playlist_id)

In [ ]:
# get_track_embedding('spotify:track:0uqPG793dkDDN7sCUJJIVC').shape
get_playlist_embedding(0)[0].shape

In [ ]:
_dot = np.inner

def cal_similarity(playlist_id_1, playlist_id_2):
    _, normed_a = cached_playlist_embedding[playlist_id_1]
    _, normed_b = cached_playlist_embedding[playlist_id_2]
    return _dot(normed_a, normed_b)
        

In [ ]:
%%timeit
cal_similarity(1, 5)

In [ ]:
def biased_w2v_random_walk_playlist(query,g,dropped_track_id, beta = 1000):
    
    N = 10000
#     totSteps is the number of total steps where step is going from playlist to track to playlist
    totSteps = 0
#     nHighVisited is the number of track nodes that have been visited at least nv times
    nHighVisited = 0
    nv = 5
#     nt is the least number of tracks which have been visited at least nv times
    nt = 200
    num_visits = defaultdict(int)

    weight_cache = {}
    def cal_weight(edge):
        if edge in weight_cache:
            return weight_cache[edge]

        val = cal_similarity(query,edge)
        weight_cache[edge] = val
        return val

    while totSteps < N and nHighVisited < nt: 
        currPlaylist = query
        # Number of iterations
        currSteps = 5
        for i in range(currSteps):
            # takes 1 step (from a playlist to track)
            edges = list(g[currPlaylist])
            if currPlaylist == query:
                edges.remove(dropped_track_id)
            currTrack = random.choice(edges)

            # takes 1 step (from a track to playlist)
            edges = list(g[currTrack])
            if currTrack == dropped_track_id:
                edges.remove(query)

            weights = [cal_weight(edge) for edge in edges]
            weights_np = np.asarray(weights)

# see https://stackoverflow.com/questions/34968722/how-to-implement-the-softmax-function-in-python
#             # this bit makes it more stable numerically
            max_weight = np.max(weights_np)
            weights_np -= max_weight
            # but does not change the answer
            weights_np = np.exp(beta * weights_np)

            prob_dis = weights_np / float(sum(weights_np))
            currPlaylist = int(choice(edges, 1, p=prob_dis))
            num_visits[currTrack] += 1
            if num_visits[currTrack] == nv:
                nHighVisited += 1
        totSteps += currSteps
    return num_visits

In [ ]:
g, tracks_dict, tracks_id_dict = playlist.make_graph_dict(playlist_list)

In [ ]:
def cal_most_visited_playlist(playlist_list, playlist_id):
    dropped_track_id = random.choice(tuple(g[playlist_id]))

    if len(g[playlist_id]) < 5:
        return None, None
    num_visits = biased_w2v_random_walk_playlist(playlist_id, g, dropped_track_id)
    suggested_ids = sorted(num_visits, key=num_visits.get, reverse=True)
    return dropped_track_id, suggested_ids

In [ ]:

all_true_ids, all_suggested_ids = playlist.process_playlists(cal_most_visited_playlist,
                                                             playlist_list)

In [ ]:

np.savez('ids_w2v_biased_random_walk-Playlist_to_playlist_N10k',
         all_true_ids=all_true_ids, all_suggested_ids=all_suggested_ids)

In [ ]:
results = playlist.cal_results(playlist_list, all_true_ids, all_suggested_ids)
filename = 'results%d_w2v_biased_random_walk-Playlist_to_playlist_N10k.txt' % (len(playlist_list),)
with open(filename, 'w') as output:
    output.write(str(results))

In [ ]:
import matplotlib.pyplot as plt

weights = [cal_similarity(0, 1+i) for i in range(10)]
weights_np = np.asarray(weights)
prob_dis = weights_np / float(sum(weights_np))
plt.plot(prob_dis, label="flat")
for beta in [1,1000,10000,100000]:
    weights_np = np.asarray(weights)
    # this bit makes it more stable numerically
    max_weight = np.max(weights_np)
    weights_np -= max_weight
    # but does not change the answer
    weights_np = np.exp(beta * weights_np)

    prob_dis = weights_np / float(sum(weights_np))
    plt.plot(prob_dis, label="beta=" + str(beta))
plt.legend()
plt.show()